In [1]:
import tensorflow as tf
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

#blas gemm launch failed
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=configuration)

In [2]:
def load_dataset():
    dataset = pd.read_csv('./lie-dataset.csv')
    # print(dataset)
    features = dataset[['HeartRate', 'BlinkFreq', 'EyeContact']]
    # print(features)
    targets = dataset[['Class']]

    features = MinMaxScaler().fit_transform(features)
    targets = OneHotEncoder(sparse=False).fit_transform(targets)

    return train_test_split(features, targets, test_size = 0.2)

input_train, input_test, output_train, output_test = load_dataset()

# Feed forward
def feed_forward():
    weightxbias1 = tf.matmul(features_placeholder, weight['input_to_hidden_one']) + bias['input_to_hidden_one']
    activation1 = tf.nn.sigmoid(weightxbias1)

    weightxbias2 = tf.matmul(activation1, weight['hidden_one_to_hidden_two']) + bias['hidden_one_to_hidden_two']
    activation2 = tf.nn.sigmoid(weightxbias2)

    weightxbias3 = tf.matmul(activation2, weight['hidden_two_to_output']) + bias['hidden_two_to_output']
    activation3 = tf.nn.sigmoid(weightxbias3)

    return activation3

In [3]:
# Architecture
layer = {
    'input': 3,
    'hidden_one': 6,
    'hidden_two': 3,
    'output': 2
}

# Stochastice Weight and Bias
weight = {
    'input_to_hidden_one': tf.Variable(tf.random.normal([layer['input'], layer['hidden_one']])),
    'hidden_one_to_hidden_two': tf.Variable(tf.random.normal([layer['hidden_one'], layer['hidden_two']])),
    'hidden_two_to_output': tf.Variable(tf.random.normal([layer['hidden_two'], layer['output']]))
}

bias = {
    'input_to_hidden_one': tf.Variable(tf.random.normal([layer['hidden_one']])),
    'hidden_one_to_hidden_two': tf.Variable(tf.random.normal([layer['hidden_two']])),
    'hidden_two_to_output': tf.Variable(tf.random.normal([layer['output']]))
}

# Set placeholder
features_placeholder = tf.placeholder(tf.float32, [None, layer['input']])
targets_placeholder = tf.placeholder(tf.float32, [None, layer['output']])

# Define epoch and leaning rate
epoch = 5000
learning_rate = 0.1

# Calculate loss + optimze
output = feed_forward()
loss = tf.reduce_mean(0.5 * (output - targets_placeholder) ** 2)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    feed = {
        features_placeholder: input_train,
        targets_placeholder: output_train
    }

    # Train
    for i in range(1, epoch + 1):
        sess.run(train, feed_dict = feed)

        if(i % 200 == 0):
            print("Epoch: %d, Loss: %.2f%%"%(i, sess.run(loss, feed_dict = feed)))

    # Test

    feed = {
        features_placeholder: input_test,
        targets_placeholder: output_test
    }
    match = tf.equal(tf.argmax(output, axis = 1), tf.argmax(targets_placeholder, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(match, tf.float32))

    print("Accuracy: {}%".format(sess.run(accuracy, feed_dict = feed) * 100))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch: 200, Loss: 0.17%
Epoch: 400, Loss: 0.15%
Epoch: 600, Loss: 0.13%
Epoch: 800, Loss: 0.13%
Epoch: 1000, Loss: 0.12%
Epoch: 1200, Loss: 0.12%
Epoch: 1400, Loss: 0.12%
Epoch: 1600, Loss: 0.12%
Epoch: 1800, Loss: 0.12%
Epoch: 2000, Loss: 0.12%
Epoch: 2200, Loss: 0.12%
Epoch: 2400, Loss: 0.12%
Epoch: 2600, Loss: 0.12%
Epoch: 2800, Loss: 0.12%
Epoch: 3000, Loss: 0.11%
Epoch: 3200, Loss: 0.11%
Epoch: 3400, Loss: 0.10%
Epoch: 3600, Loss: 0.10%
Epoch: 3800, Loss: 0.09%
Epoch: 4000, Loss: 0.08%
Epoch: 4200, Loss: 0.07%
Epoch: 4400, Loss: 0.06%
Epoch: 4600, Loss: 0.06%
Epoch: 4800, Loss: 0.05%
Epoch: 5000, Loss: 0.04%
Accuracy: 100.0%
